## Ensembling & Stacking models

In [2]:
import pickle
import pandas as pd
import numpy as np
import re
import os
import warnings
warnings.filterwarnings('ignore')

import sklearn
import xgboost as xgb
# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier,
                              AdaBoostClassifier,
                              GradientBoostingClassifier,
                              ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.model_selection import KFold

# import plotly.offline as py
# py.init_notebook_mode(connected=True)
# import plotly.graph_objs as go
# import plotly.tools as tls

In [4]:
with open('./ckpt/train_proc.pkl', mode='rb') as f:
    train = pickle.load(f)
with open('./ckpt/test_proc.pkl', mode='rb') as f:
    test = pickle.load(f)
with open('./ckpt/test_passengerid.pkl', mode='rb') as f:
    passenger = pickle.load(f)

In [6]:
# Some useful parametes which will come in handy later on
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0
NFOLDS = 5
kf = KFold(n_splits=NFOLDS, random_state=SEED)

In [7]:
# Class to extend the sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)
        
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
        
    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self, x, y):
        return self.clf.fit(x, y)
    
    def feature_importances(self, x, y):
        print(self.clf.fit(x, y).feature_importances_)
        
# Class to extend XGboost classifier